## Loading dataset 

In [2]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [3]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 93% 178M/191M [00:00<00:00, 198MB/s]
100% 191M/191M [00:01<00:00, 197MB/s]


In [4]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [5]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 14.1 MB/s 
     |████████████████████████████████| 1.2 MB 92.8 MB/s 
     |████████████████████████████████| 1.1 MB 79.7 MB/s 
     |████████████████████████████████| 120 kB 91.7 MB/s 
     |████████████████████████████████| 596 kB 82.9 MB/s 
     |████████████████████████████████| 6.4 MB 60.5 MB/s 
     |████████████████████████████████| 840 kB 77.4 MB/s 
     |████████████████████████████████| 87 kB 8.5 MB/s 
     |████████████████████████████████| 3.4 MB 74.4 MB/s 
     |████████████████████████████████| 1.1 MB 70.2 MB/s 
     |████████████████████████████████| 234 kB 69.1 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 352 kB 92.9 MB/s 
     |████████████████████████████████| 99 kB 13.5 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 462 kB 88.2 MB/s 
     |█████████████

In [6]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [7]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [8]:

spec = model_spec.get('efficientdet_lite0')

In [9]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [10]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [11]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/10
176/176 [==============================] - 76s 211ms/step - det_loss: 1.2365 - cls_loss: 0.9246 - box_loss: 0.0062 - reg_l2_loss: 0.0640 - loss: 1.3005 - learning_rate: 0.0071 - gradient_norm: 2.7314 - val_det_loss: 0.8084 - val_cls_loss: 0.7132 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0640 - val_loss: 0.8725
Epoch 2/10
176/176 [==============================] - 35s 200ms/step - det_loss: 0.7071 - cls_loss: 0.5923 - box_loss: 0.0023 - reg_l2_loss: 0.0641 - loss: 0.7712 - learning_rate: 0.0058 - gradient_norm: 3.0898 - val_det_loss: 0.6401 - val_cls_loss: 0.5414 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0641 - val_loss: 0.7043
Epoch 3/10
176/176 [==============================] - 36s 202ms/step - det_loss: 0.5767 - cls_loss: 0.4905 - box_loss: 0.0017 - reg_l2_loss: 0.0642 - loss: 0.6409 - learning_rate: 0.0051 - gradient_norm: 2.9896 - val_det_loss: 0.5626 - val_cls_loss: 0.5366 - val_box_loss: 5.1992e-04 - val_reg_l2_loss: 0.0642 - val_loss: 0.6268
Epoch 4/10
176/176 [=

## Evaluating the model 

In [12]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 12s 178ms/step



{'AP': 0.4827312,
 'AP50': 0.5793742,
 'AP75': 0.5633904,
 'AP_/Casual Shirt': 0.41627264,
 'AP_/Desgin sherwani': 0.61951107,
 'AP_/Design Kurta': 0.21998873,
 'AP_/Formal Shirt': 0.43771073,
 'AP_/Plain kurta': 0.24873042,
 'AP_/Plain sherwani': 0.61818707,
 'AP_/tshirt': 0.8187177,
 'APl': 0.48275006,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.87229556,
 'ARm': -1.0,
 'ARmax1': 0.8382318,
 'ARmax10': 0.87229556,
 'ARmax100': 0.87229556,
 'ARs': -1.0}

## Saving Model 

In [14]:
model.export(export_dir='/content/')